# Install and Import Dependencies

In [4]:
!pip install torch torchvision torchaudio
!pip install transformers requests beautifulsoup4 pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.9/278.9 kB 6.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.6 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 9.4 MB/s eta 0:00:0000:01


In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

# Instantiate Model

In [7]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/Users/yehiafahmy/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Basic Test

In [30]:
tokens1 = tokenizer.encode('This was a decent place to eat', return_tensors='pt')

In [31]:
result = model(tokens1)
int(torch.argmax(result.logits))+1

4

# Collect Reviews

In [33]:
r = requests.get('https://www.yelp.com/biz/kinkaku-izakaya-kitchener?osq=Restaurants')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [43]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [52]:
def get_sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

# Calculate Sentiment of the Reviews

In [65]:
df['sentiment'] = df['review'].apply(lambda x: get_sentiment_score(x[:512]))
df

,review,sentiment
0,My BF is an alum of UWaterloo and said this wa...,4
1,Waterloo has some hidden sushi places. I would...,3
2,We were here twice once for the dine in ayce a...,3
3,Visiting from another country -- loved the fac...,5
4,"I loveeeee the food here, the ratio of salmon ...",5
5,"Honestly, this was my go-to place for sushi du...",4
6,I'm I would say it was quite disappointing to ...,2
7,I was visiting my friend in Waterloo and for d...,5
8,Maybeeee the best sushi I ever had and I've ha...,5
9,This place is the best All You Can Eat (AYCE) ...,5
